[View in Colaboratory](https://colab.research.google.com/github/Curiousss/FLD/blob/master/FLDNetwork.ipynb)

In [1]:
!wget https://www.dropbox.com/s/9j2om0gcv43hkz8/300w_cropped.zip

--2018-09-03 09:14:54--  https://www.dropbox.com/s/9j2om0gcv43hkz8/300w_cropped.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/9j2om0gcv43hkz8/300w_cropped.zip [following]
--2018-09-03 09:14:55--  https://www.dropbox.com/s/raw/9j2om0gcv43hkz8/300w_cropped.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc758bc09139f4c6de8b13cfa14c.dl.dropboxusercontent.com/cd/0/inline/APkar3lnIC2_Z1Xz-Lborw2Pdmuw8co9w65xrO_klABVMih_wETsyeQsbKsdNrf-1_U2-_jDTwn6uB3TFXD3OtSRN-d7NcYJd8U9ing6nwVxn-7wM62f0HDOsR-4C38zAbsZzmotz5DE50-HygwjKGlZu5ZTymkkFsnq-QBz1mg7hjPJgkAlrqp4GPNYmBV23Aw/file [following]
--2018-09-03 09:14:55--  https://uc758bc09139f4c6de8b13cfa14c.dl.dropboxusercontent.com/cd/0/inline/APkar3lnIC2_Z1Xz-Lborw2Pdmuw8co9w65x

In [0]:
!unzip 300w_cropped.zip

In [2]:
!ls

300w_cropped.zip  sample_data


In [4]:
from google.colab import files
uploaded = files.upload()

Saving kernel45.txt to kernel45.txt


In [5]:
!git clone https://github.com/albanie/pts_loader
!ls pts_loader/
!cp pts_loader/pts_loader.py .
!ls

Cloning into 'pts_loader'...
remote: Counting objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16
Unpacking objects: 100% (16/16), done.
LICENSE.md  pts_loader.py  README.md  tests
01_Indoor   300w_cropped.zip  pts_loader     sample_data
02_Outdoor  kernel45.txt      pts_loader.py


In [0]:
import os
from os import listdir,makedirs
from os.path import isfile,join
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import tensorflow as tf
import keras.backend as K
import keras
from keras.models import Sequential
import cv2

from keras.layers import Dense, Dropout, Activation, Add, SeparableConv2D
from keras.layers import Convolution2D, Lambda, Input, Reshape
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.layers import LeakyReLU



In [0]:
'''
Hyper-Parameters
'''
image_H = 96.0
image_W = 96.0
grayscale = True
batch_size = 16
learning_rate=0.001
momentum = 0
decay = 0
epochs = 100
droupout= 0
gradient_clipping = 0.1
nFilters = 128
nlandmarks = nLabels = 68
K=45
img_rows = 96
img_cols = 96
img_channels = 1
alpha = 0.1

In [0]:
# Get files
import pts_loader

img_x = []
pts_y = []
for i in range(1,300):
    #print("iteration", i)
    if i == 78 or i == 79 or i == 266:
      continue
    if i < 10:
        img_f = r"01_Indoor/indoor_00" + str(i) +".png"
        pts_f = r"01_Indoor/indoor_00" + str(i) +".pts"
    elif i < 100:
        img_f = r"01_Indoor/indoor_0" + str(i) +".png"
        pts_f = r"01_Indoor/indoor_0" + str(i) +".pts"
    else:
        img_f = r"01_Indoor/indoor_" + str(i) +".png"
        pts_f = r"01_Indoor/indoor_" + str(i) +".pts"
    pts = pts_loader.load(pts_f)
    # Loading imgae in grayscale
    img = cv2.imread(img_f)
    
    scale_x = image_W/img.shape[0]
    scale_y = image_H/img.shape[1]
    img = cv2.resize(img, (img_rows, img_cols))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.normalize(img,None,0,255,cv2.NORM_MINMAX)
    img = img.reshape(img_rows, img_cols, 1)
    landmarks = []
    
    for p in pts:
        new_pts = []
        landmark = np.zeros((96, 96), np.uint8)
        new_pts = (p[0] * scale_y , p[1] * scale_x )
        #new_points.append(new_pts)
        x = int(new_pts[0])
        y = int(new_pts[1])
        #print(n, x, y)
        landmark[x][y] = 1 # Normalized WHite
        landmarks.append(landmark)
        #landmarks.append(x) # Testing
        #landmarks.append(y) # Testing
        
    img_x.append(img)
    pts_y.append(landmarks)


In [0]:
#Gn = apply_kernel(pts_y)

In [6]:
print(img_x[0].shape)
assert not np.any(np.isnan(img_x))
assert not np.any(np.isnan(pts_y))
train_x = []
train_y = []
test_x = []
test_y = []


(96, 96, 1)


In [0]:
train_x = np.array(img_x[:200])
train_y = np.array(pts_y[:200])
test_x = np.array(img_x[200:])
test_y = np.array(pts_y[200:])


In [8]:
print(train_y.shape)
train_y = train_y.reshape(-1, 96,96,68)
test_y = test_y.reshape(-1, 96,96,68)

print(train_y.shape)

(200, 68, 96, 96)
(200, 96, 96, 68)


In [0]:
def apply_kernel(local_out):
    '''
     get kernel from text file
     tile it to 1x1x45x45
     convolve with local out
    '''
    f = open(r'kernel45.txt')
    kernel45 = np.fromfile(f, dtype = np.float32, count = -1, sep = " ")
    
    # [1 x 1 x 45 x 45]
    kernel45 = np.reshape(kernel45, (45, 45, 1, 1))
    # [numLabels x 1 x 45 x 45] if tensor convolution is used
    kernel_stacked 	= np.tile(kernel45, (1, 1, nLabels, 1))
    kernel_tensor = tf.convert_to_tensor(kernel_stacked, dtype=tf.float32)
    print("local_out", local_out)
    print("kernel", kernel_tensor)

    groups = []
    
    for j in range(nLabels):
        groups.append(tf.nn.conv2d(local_out, kernel_tensor, strides=[1,1,1,1],
                                   padding="SAME"))
    
    heatmaps = tf.concat(groups, 0)
    heatmaps = tf.reshape(heatmaps, (-1, 96, 96, 68))
    print("Heatmaps", heatmaps)
    return heatmaps



In [20]:
from keras.layers import LeakyReLU
'''
----------------------------------------LOCAL SUBNET----------------------------
'''

# [ImageC x ImageH x ImageW]
#-----------------------------------------------5 layers of kernel 5x5----------
input_img = Input(( img_rows, img_cols, img_channels))
model = SeparableConv2D(filters=nFilters, kernel_size=(5, 5), padding='same')(input_img)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)

model = SeparableConv2D(filters=nFilters, kernel_size=(5, 5), padding='same')(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)

model = SeparableConv2D(filters=nFilters, kernel_size=(5, 5), padding='same')(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)

model = SeparableConv2D(filters=nFilters, kernel_size=(5, 5), padding='same')(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)

model = SeparableConv2D(filters=nFilters, kernel_size=(5, 5), padding='same')(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)

#-----------------------------------------------10 layers of kernel 3x3----------
#1
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same')(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)
#2
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same')(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)
#3
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same')(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)
#4
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same')(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)
#5
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same')(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)
#6
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same')(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)
#7
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same')(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)
#8
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same')(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)
#9
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same')(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)
#10
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same')(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)

print("Local subnet", model)

'''
----------------------1x1 Convolution with activation-----------------------------
'''
model_Conv = Convolution2D(filters=nLabels, kernel_size=(1, 1))(model)
model = LeakyReLU(alpha)(model)
model_Conv = Lambda(apply_kernel, output_shape=(img_cols, img_rows, nLabels))(model_Conv)
print("Main 1x1", model_Conv)
'''
-------------------------------------Ol 1x1 Linear convolution-------------------
Apply Gaussian Kernel filter
'''
model_Ol = Convolution2D(filters=nLabels, kernel_size=(1, 1),activation=None)(model)
model_Ol = Lambda(apply_kernel, output_shape=(img_cols, img_rows, nLabels), name='Ol')(model_Ol)
print("Local output", model_Ol)

output_Ol = keras.layers.concatenate([model_Conv, model_Ol])
print("Concatenate", output_Ol)
output_Ol = Reshape((96, 96, -1))(output_Ol)
#print("Reshape", output_Ol)

'''
----------------------------------------GLOBAL SUBNET----------------------------
'''
#--------------------------------7 layers of dialated kernel 3x3------------------
#1
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4)(output_Ol)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)
#2
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4)(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)
#3
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4)(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)
#4
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4)(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)
#5
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4)(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)
#6
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4)(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)
#7
model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4)(model)
model = BatchNormalization()(model)
model = LeakyReLU(alpha)(model)
model = Dropout(droupout)(model)

model = SeparableConv2D(filters=nLabels, kernel_size=(1, 1), activation=None)(model)

FLD_model = Model(inputs = input_img, outputs=model)
FLD_model.summary()


Local subnet Tensor("dropout_37/Identity:0", shape=(?, 96, 96, 128), dtype=float32)
local_out Tensor("conv2d_5/BiasAdd:0", shape=(?, 96, 96, 68), dtype=float32)
kernel Tensor("lambda_3/Const:0", shape=(45, 45, 68, 1), dtype=float32)
Heatmaps Tensor("lambda_3/Reshape:0", shape=(?, 96, 96, 68), dtype=float32)
Main 1x1 Tensor("lambda_3/Reshape:0", shape=(?, 96, 96, 68), dtype=float32)
local_out Tensor("conv2d_6/BiasAdd:0", shape=(?, 96, 96, 68), dtype=float32)
kernel Tensor("Ol_2/Const:0", shape=(45, 45, 68, 1), dtype=float32)
Heatmaps Tensor("Ol_2/Reshape:0", shape=(?, 96, 96, 68), dtype=float32)
Local output Tensor("Ol_2/Reshape:0", shape=(?, 96, 96, 68), dtype=float32)
Concatenate Tensor("concatenate_3/concat:0", shape=(?, 96, 96, 136), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 9

In [0]:
import tensorflow as tf
from keras import backend as K

def landmark_weight(labels):
    
    #[numLabels x 1 x 1]
    W1 = tf.reduce_sum(labels, axis=2)
    W2 = tf.reduce_sum(W1, axis=1)
    W3 = tf.reduce_mean(W2)
    W4 = tf.multiply(W3, 0.5)
    Win = tf.add(W4, W2)
    print("Win", Win.shape)
    Win = tf.reshape(Win, (-1, 1, 1, nLabels))
    return Win

'''
Ol - [numLabels x ImageH x ImageW] Kernel conv already applied in Network
Og - [numLabels x ImageH x ImageW]
labels - Landmark coordinates [numLabels x ImageH x ImageW] without
kernel applied
'''
def loss_function(labels, Og):
    
    #Gn = labels #testing
    # Ground Truth Heatmap
    Gn = apply_kernel(labels)

    Win = landmark_weight(labels)
    #Win = 1 #testing
    print("---------------Loss Function----------------")
    print("Og", Og)
    print("Labels", labels)
    print("Ground Truth", Gn)
    print("Ol", model_Ol)
    # (Og - Gn)2
    global_diff = tf.subtract(Og, Gn)
    global_err = tf.square(global_diff)
    
    # (Ol - Gn)2
    
    #How to access intermediate layer or model from here???
    #Currently using global variable model_Ol
    
    # Kernel is already applied for model_Ol
    local_diff = tf.subtract(model_Ol, Gn)
    local_err = tf.square(local_diff)
    
    comb_err = tf.add(local_err, global_err)
    comb_Win = tf.transpose(Win) * tf.transpose(comb_err)
    wt_square_err = tf.reduce_sum(comb_Win)
    wt_square_err = tf.multiply(wt_square_err, 0.05)
    print("Loss", wt_square_err)
    #return wt_square_err    
    
    return tf.reduce_mean(comb_err) #testing


In [25]:
from keras import optimizers
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=0.1)
FLD_model.compile(optimizer=sgd,
              loss=loss_function,
              metrics=['accuracy'])


local_out Tensor("separable_conv2d_69_target_1:0", shape=(?, ?, ?, ?), dtype=float32)
kernel Tensor("loss_3/separable_conv2d_69_loss/Const:0", shape=(45, 45, 68, 1), dtype=float32)
Heatmaps Tensor("loss_3/separable_conv2d_69_loss/Reshape:0", shape=(?, 96, 96, 68), dtype=float32)
Win (?, ?)
---------------Loss Function----------------
Og Tensor("separable_conv2d_69/BiasAdd:0", shape=(?, 96, 96, 68), dtype=float32)
Labels Tensor("separable_conv2d_69_target_1:0", shape=(?, ?, ?, ?), dtype=float32)
Ground Truth Tensor("loss_3/separable_conv2d_69_loss/Reshape:0", shape=(?, 96, 96, 68), dtype=float32)
Ol Tensor("Ol_2/Reshape:0", shape=(?, 96, 96, 68), dtype=float32)
Loss Tensor("loss_3/separable_conv2d_69_loss/Mul_2:0", shape=(), dtype=float32)


In [26]:
 FLD_model.fit(train_x, train_y, batch_size=8, epochs=10, validation_data=(test_x, test_y))

Train on 200 samples, validate on 96 samples
Epoch 1/10
200/200 [==============================] - 70s 350ms/step - loss: 147108.8505 - acc: 0.0046 - val_loss: 15674.4963 - val_acc: 3.5378e-04
Epoch 2/10
200/200 [==============================] - 62s 309ms/step - loss: 3533.7674 - acc: 0.0039 - val_loss: 29187.9956 - val_acc: 2.7353e-04
Epoch 3/10
200/200 [==============================] - 62s 309ms/step - loss: 1917.6627 - acc: 0.0038 - val_loss: 16792.1503 - val_acc: 5.4140e-04
Epoch 4/10
200/200 [==============================] - 62s 309ms/step - loss: 1297.2790 - acc: 0.0037 - val_loss: 13474.6290 - val_acc: 4.8941e-04
Epoch 5/10
200/200 [==============================] - 62s 309ms/step - loss: 1053.6581 - acc: 0.0038 - val_loss: 12602.6707 - val_acc: 5.3462e-04
Epoch 6/10
200/200 [==============================] - 62s 309ms/step - loss: 985.0135 - acc: 0.0031 - val_loss: 8339.6291 - val_acc: 8.8727e-04
Epoch 7/10
200/200 [==============================] - 62s 309ms/step - loss: 84

In [0]:
import cntk as C

cntk_model = C.load_model(r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\Merget\Merget\python\output\models\model.21")

In [0]:
cntk_model.parameters

In [0]:
cntk_model.save("cntk_model.h5")

In [0]:
FLD = keras.models.load_model(r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\FLDCode\cntk_model.h5")

In [0]:
import keras.backend as K

#save_model_hdf5(cntk_model, 'my_model.h5')
kerasmodel = K.functional_ops. load_model(cntk_model)

In [0]:
#func = cntk_model.outputs[0]
cntk_model.save('cntk_model.onnx', format=C.ModelFormat.)

In [0]:
import onnx
onnx_model = onnx.load('cntk_model.onnx')

In [0]:
from onnx_tf.backend import prepare
tf_model = prepare(onnx_model)